In [1]:
from azureml.core import Workspace, Experiment, Dataset, Datastore

ws = Workspace.from_config()

In [3]:
BLOB_CONTAINER = os.environ.get("BLOB_CONTAINER_NAME", "diabetes")
BLOB_ACCOUNT = os.environ.get("BLOB_ACCT_NAME", "publicmldatasc")
BLOB_SAS_TOKEN = os.environ.get("BLOB_SAS_TOKEN", "?si=DiabetesReadOnly&sv=2019-10-10&sr=c&sig=cz9P%2B1V1eC6FvDIKBQNmA5nWqbsGfkqzdPTTYmiidfg%3D")

In [4]:
datastore_name = 'diabetes'

from azureml.core import Datastore

if ws.datastores.get(datastore_name):
    print(f"'{datastore_name}' datastore found. Skipping registration")
else:
    print(f"'{datastore_name}' datastore not found. Registering with Workspace")
    _ = Datastore.register_azure_blob_container(workspace=ws,
                                                datastore_name='diabetes',
                                                container_name=BLOB_CONTAINER,
                                                account_name=BLOB_ACCOUNT,
                                                sas_token=BLOB_SAS_TOKEN)

'diabetes' datastore found. Skipping registration


In [5]:
ds = ws.datastores['diabetes']

In [7]:
from azureml.core import Dataset

dataset = Dataset.Tabular.from_parquet_files(path=(ds, 'parquet/diabetes.parquet'))
dataset = dataset.register(ws, name="diabetes")

In [6]:
file_dataset = Dataset.File.from_files(path=(ds, 'parquet/diabetes.parquet'))
file_dataset = file_dataset.register(ws, name="diabetes-files")

## Create XGBoost Environment

In [19]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environ = Environment(name="MLFlow-XGBoost")
environ.docker.enabled = True

environ.python.conda_dependencies = CondaDependencies.create(python_version="3.8.6",
                                                             conda_packages=["pip"],
                                                             pip_packages=["mlflow==1.11.0", 
                                                                           "xgboost==1.2.1", 
                                                                           "pandas==1.1.4", 
                                                                           "scikit-learn==0.23.2",
                                                                           "azureml-mlflow==1.17.0.post1",
                                                                           "azureml-defaults==1.17.0",
                                                                           "matplotlib==3.3.2"], 
                                                             pin_sdk_version=True)

In [20]:
environ = environ.register(ws)
build = environ.build(ws)
build.wait_for_completion(show_output=True)